# Trainingsdaten Erstellen 

Im Folgenden werden die Rohdaten aufbereitet. Dazu werden die Bilder verkleinert und mit Hilfe von Data Augmentation variiert. Danach werden die mit LabelImg erstellten Annotationen in ein Trainings- und ein Testdatenset umgewandelt.


In [1]:
!pip install -U git+https://github.com/albu/albumentations

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-81dnal3g
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-81dnal3g
     |████████████████████████████████| 634kB 52.0MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65100 sha256=dc8980be060f98e441e06ea1f519b66956be7b321750559282ba069c75a5f7bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-ty30plvp/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=076dbbea32be9a0f6e8cd7501f36fdc0df4b6bb5a6968bce63cf3d70f4c6dcd5
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
 

## Benötigte Bibliotheken

In [ ]:
from __future__ import division

import os
import glob
import numpy as np
import cv2
from matplotlib import pyplot as plt
from IPython.display import display, HTML 

from albumentations import *
from google.colab import drive

## Rohdaten mouten

Am einfachsten ist es, wenn man die Daten auf einem Cloud-Storage gespeichert hat. Für das Tutorial kommt Google Drive zum Einsatz. Auf diesem sind alle Bilder im Rohformat im Verzeichnis *ix-tut-raw* gespeichert. In die virtuelle Maschine von Colab lässt sich dieses Verzeichnis über einen einfachen **mount** Befehl freigeben mit 

In [3]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Der Inhalt des so gemounteten Google Drive Verzeichnisses lässt sich mit **ls** einfach wie ein lokales Verzeichnis ausgeben.

In [50]:
!ls -la 'drive/My Drive/data/ix-tut-raw' 

total 100679
-rw------- 1 root root 2234786 Mar 27 10:13 c3po_0001.jpg
-rw------- 1 root root 2111258 Mar 27 10:13 c3po_0002.jpg
-rw------- 1 root root 2150837 Mar 27 10:14 c3po_0003.jpg
-rw------- 1 root root 2014113 Mar 27 10:14 c3po_0004.jpg
-rw------- 1 root root 2122045 Mar 27 10:14 c3po_0005.jpg
-rw------- 1 root root 2203209 Mar 27 10:14 c3po_0006.jpg
-rw------- 1 root root 2166206 Mar 27 10:15 c3po_0007.jpg
-rw------- 1 root root 2079133 Mar 27 10:15 c3po_0008.jpg
-rw------- 1 root root 2121983 Mar 27 10:15 c3po_0009.jpg
-rw------- 1 root root 2172382 Mar 27 10:15 c3po_0010.jpg
-rw------- 1 root root 2391137 Mar 27 10:05 luke-skywalker_0001.jpg
-rw------- 1 root root 2002863 Mar 27 10:05 luke-skywalker_0002.jpg
-rw------- 1 root root 2053317 Mar 27 10:06 luke-skywalker_0003.jpg
-rw------- 1 root root 1907511 Mar 27 10:06 luke-skywalker_0004.jpg
-rw------- 1 root root 1838716 Mar 27 10:06 luke-skywalker_0005.jpg
-rw------- 1 root root 1977107 Mar 27 10:06 luke-skywalker_0006.jpg

## Variablen

Definition der Eingabe- und Ausgabeverzeichnisse. Das Ausgabeverzeichnis liegt auch wieder auf dem Google Drive. Zu beachten ist bei der Verwendung von Cloud Storage, dass die Synchronisation der Daten etwas dauern kann.

In [51]:
# Eigene Fotos
raw_images = '/content/drive/My Drive/data/ix-tut-raw/'

# Ausgabeverzeichnis
processed_images = '/content/drive/My Drive/data/ix-tut-processed/'

# Alle Bilddateien mit der Endung jpg
raw_image_lst = glob.glob(raw_images+'*.jpg')

print(f"Liste enthält {len(raw_image_lst)} Bilder")

Liste enthält 50 Bilder


## Funktionen

Ein paad Hilfsfunktionen für die Bildverarbeitung

In [ ]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)

    return resized

def processed_filename(image_filename):
    filepath, filename_ext = os.path.split(image_filename)
    filename, file_ext = os.path.splitext(filename_ext)
    image_number = filename.split("_")[1]

    return filename, image_number, file_ext

## Alle Bilder modifizieren

Die Ergebnisbilder heißen wie das Originalbild plus einer fortlaufenden Nummer

In [53]:
filename, image_number, file_ext = processed_filename(raw_image_lst[0])
print(filename)
print(image_number)
print(file_ext)

luke-skywalker_0001
0001
.jpg


## Schleife über alle Bilder

In diesem Schritt wird die ursprüngliche Bildgröße reduziert und das Bild wird in das für das YOLOv3 neuronale Netz optimierte Format 416x416 (13*32) gebracht. Pro Bild werden dann 13 unterschiedliche Bildverarbeitungsschritte angewendet um die nötigen Bildvariationen zu erzeugen.

In [54]:
images = 0
idx = 0
proc_imgs_dict = {}
show = True

for image in raw_image_lst:
  idx = idx + 1
  print(f"Verarbeite {idx}/{len(raw_image_lst)} Bild '{image}...")
  
  # Bild einlesen
  img = cv2.imread(image)
  
  # Data Augmentation 1: Größenänderung (416x416) 
  img1 = image_resize(img, height=416)
  aug = CenterCrop(416,416, p=1.0)
  img1 = aug.apply(img1)

  aug_name = "resize"
  processed_image_filename, image_number, file_ext = processed_filename(image)
  proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
  print(f"Store {proc_filename}")
  cv2.imwrite(proc_filename, img1)
  proc_imgs_dict[aug_name] = img1
  images = images + 1

  # Data Augmentation 2: Vertikale Spiegerlung
  idx = idx + 1
  aug = VerticalFlip(p=1)
  aug_name = "vertflip"
  img2 = aug.apply(img1)
  proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}" 
  print(f"Store {proc_filename}")
  cv2.imwrite(proc_filename, img2)
  proc_imgs_dict[aug_name] = img2
  images = images + 1

  # Data Augmentation 3: Horizontale Spiegelung
  idx = idx + 1
  aug = HorizontalFlip(p=1)
  aug_name = "horzflip"
  img3 = aug.apply(img1)
  proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
  print(f"Store {proc_filename}")
  cv2.imwrite(proc_filename, img3)
  proc_imgs_dict[aug_name] = img3
  images = images + 1

  # Data Augmentation 4: Elatische Transformation
  idx = idx + 1
  aug = ElasticTransform(alpha=101, sigma=81, alpha_affine=53, p=0.5)
  aug_name = "elastic"
  img4 = aug.apply(img1)
  proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
  print(f"Store {proc_filename}")
  cv2.imwrite(proc_filename, img4)
  proc_imgs_dict[aug_name] = img4
  images = images + 1

  # Data Augmentation 5: Rotation
  idx = idx + 1
  aug = RandomRotate90(p=1.0)
  aug_name = "rot"
  img5 = aug.apply(img1, factor=45)
  proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
  print(f"Store {proc_filename}")
  cv2.imwrite(proc_filename, img5)
  proc_imgs_dict[aug_name] = img5
  images = images + 1

  for proc_name in proc_imgs_dict:
    idx = 0
    
    # Data Augmentation 6: Alphawert ändern
    idx = idx + 1
    alpha = 2.2
    aug = RandomContrast(limit=0.9, p=1.0)
    aug_name = "alpha"
    img6 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img6)
    images = images + 1

    # Data Augmentation 7: Zufällige Helligkeit
    idx = idx + 1
    alpha = 1.2
    aug = RandomBrightness(limit=0.2, p=1.0)
    aug_name = "bright"
    img7 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img7)
    images = images + 1
    
    # Data Augmentation 8: Grauwerte
    idx = idx + 1
    aug = ToGray(p=0.5)
    aug_name = "gray"
    img8 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img8)
    images = images + 1

    # Data Augmentation 9: Sepia
    idx = idx + 1
    aug = ToSepia(p=1)
    aug_name = "sepia"
    img9 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img9)
    images = images + 1

    # Data Augmentation 10: Weichzeichner
    idx = idx + 1
    aug = Blur(blur_limit=11, p=1)
    aug_name = "blur"
    img10 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img10)
    images = images + 1

    # Data Augmentation 11: Noise
    idx = idx + 1
    aug = MultiplicativeNoise(always_apply=True, elementwise=True, multiplier=(0.9, 1.1), p=1.0)
    aug_name = "noise"
    img11 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img11)
    images = images + 1

    # Data Augmentation 12: JPEGCompression
    idx = idx + 1
    aug = JpegCompression(quality_lower=0, quality_upper=1, p=1)
    aug_name = "jpeg"
    img12 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img12)
    images = images + 1

    # Data Augmentation 13: Farbkanal
    idx = idx + 1
    aug = ChannelDropout(channel_drop_range=(1, 1), fill_value=0, p=1)
    aug_name = "channel"
    img13 = aug.apply(proc_imgs_dict[proc_name])
    proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
    print(f"Store {proc_filename}")
    cv2.imwrite(proc_filename, img13)
    images = images + 1
    
  # Bilderindex zurücksetzen
  idx = 0

print("{} Bilder erzeugt".format(images))


Verarbeite 1/50 Bild '/content/drive/My Drive/data/ix-tut-raw/luke-skywalker_0001.jpg...
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_vertflip.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_horzflip.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_elastic.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_rot.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize_alpha.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize_bright.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize_gray.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize_sepia.jpg
Store /content/drive/My Drive/data/ix-tut-processed/luke-skywalker_0001_resize_blur.jpg
Store /content/drive/My Drive/data/ix-tut-processe

Es sollten sich jetzt 2250 Bildateien vorhanden sein. Für die 5 LEGO-Figuren wurden jeweils 10 Oiginalbilder mit 9 Variationen erstellt. 

In [57]:
!ls -l "/content/drive/My Drive/data/ix-tut-processed/" | wc -l

2501


## Bounding Boxes erstellen

Im nächsten Schritt müssen die Bilder annotiert werden. Da nur die fünf verschiedenen Transformationen resize, vertflip, horzflip, elatic und rot Koordinaten verändern, müssen nur diese mit Bounding Boxen versehen werden. Die anderen Annotation-Dateien erzeugen wir im Anschluß durch einfaches Kopieren.

Die Annotationen werden mit Hilfe von LabelImg erzeugt.

## Bounding Box Dateien kopieren

In [24]:
from shutil import copyfile

processed_images = '/content/drive/My Drive/data/ix-tut-annotations/'
annotated_variations = '/content/drive/My Drive/data/ix-tut-annotations-all/'
resize_processed_image_lst = glob.glob(processed_images+'*resize.xml')
vertflip_processed_image_lst = glob.glob(processed_images+'*vertflip.xml')
horzflip_processed_image_lst = glob.glob(processed_images+'*horzflip.xml')
elastic_processed_image_lst = glob.glob(processed_images+'*elastic.xml')
rot_processed_image_lst = glob.glob(processed_images+'*rot.xml')

all_processed = resize_processed_image_lst + vertflip_processed_image_lst + horzflip_processed_image_lst + elastic_processed_image_lst + rot_processed_image_lst
print(f"{len(all_processed)} to be work on")

250 to be work on


In [ ]:
variation_lst = ["alpha","bright","blur","channel","gray","jpeg","noise","sepia"] 
idx = 0
for process in all_processed:
  filename, image_number, file_ext = processed_filename(process)
  old_annotation_filename = annotated_variations + filename + file_ext
  print(f"Kopiere {process} nach {old_annotation_filename}") 
  copyfile(process, old_annotation_filename)
  idx = idx + 1
  for variation in variation_lst:
    new_annotation_filename = annotated_variations + filename + "_" + variation + file_ext
    print(f"Kopiere {process} nach {new_annotation_filename}")
    copyfile(process, new_annotation_filename)
    idx = idx + 1

print(f"{idx} Dateien erzeugt")


## YOLOv3 Anntotation Datei schreiben

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET

classes = ["r2d2","c3po","luke-skywalker","obi-wan-kinobi","sturmtruppler"]

def convert_voc_annotation(image_dir, voc_filename):
    in_file = open(voc_filename)
    tree=ET.parse(in_file)
    root = tree.getroot()

    #image_path = root.find('path').text
    #image_filename = os.path.basename(image_path)

    image_filename, image_number, file_ext = processed_filename(voc_filename)
    new_image_filename = image_dir + '/' + image_filename + '.jpg'

    yolo_line = new_image_filename
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        yolo_line += " " + ",".join([str(a) for a in b]) + ',' + str(cls_id)
        #yolo_line = image_path + " " + ",".join([str(a) for a in b]) + ',' + str(cls_id)

    return yolo_line

In [58]:
from sklearn.model_selection import train_test_split

all_files = []
annotation_path = '/content/drive/My Drive/data/ix-tut-annotations-all'
#image_path = '/content/drive/My Drive/data/ix-tut-processed'
image_path = 'ix-tut-yolov3-data/data'

voc_filenames = glob.glob(annotation_path+'/*.xml')
voc_filenames.sort()

print(f"Found {len(voc_filenames)} files...")

for voc_filename in voc_filenames:
    yolo_line = convert_voc_annotation(image_path, voc_filename)
    #print(yolo_line)
    #break
    all_files.append(yolo_line)

Found 2250 files...


## Daten in Training und Test aufteilen 

Teilen der Daten in einen Trainings- und einen Testanteil im Verhältnis 80:20

In [59]:
train_files, test_files, _, _ = train_test_split(all_files, all_files, test_size = 0.15, random_state = 0)

print(f"Anzahl Testdateien : {len(test_files)}")
print(f"Anzahl Trainingsdateien : {len(train_files)}")

Anzahl Testdateien : 338
Anzahl Trainingsdateien : 1912


## Trainings und Test-Datensätze erzeugen

Die beiden Teile werden in zwei getrennte Dateien abgespeichert

In [ ]:
!rm starwars_train.txt
!rm starwars_test.txt

In [ ]:
train_file = open('starwars_train.txt', 'w')
test_file  = open('starwars_test.txt', 'w')

for imagefile_and_box in train_files: 
   train_file.write(imagefile_and_box)
   train_file.write('\n')

train_file.close()

for imagefile_and_box in test_files: 
   test_file.write(imagefile_and_box)
   test_file.write('\n')


In [62]:
!ls -la

total 188
drwxr-xr-x 1 root root   4096 Mar 29 16:20 .
drwxr-xr-x 1 root root   4096 Mar 29 14:43 ..
drwxr-xr-x 1 root root   4096 Mar 25 16:11 .config
drwx------ 4 root root   4096 Mar 29 14:44 drive
drwxr-xr-x 1 root root   4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root  24666 Mar 29 16:20 starwars_test.txt
-rw-r--r-- 1 root root 140416 Mar 29 16:20 starwars_train.txt


In [ ]:
from google.colab import files
files.download('starwars_train.txt') 

In [ ]:
from google.colab import files
files.download('starwars_test.txt') 